In [ ]:
# Import necessary libraries
import numpy as np
import torch
from vtbench.data_utils import read_ucr, normalize_data, apply_smote, to_torch_tensors

from vtbench.models.CNN import Simple2DCNN
from vtbench.models.MLP import MLP
from vtbench.models.RNN import RNN
from vtbench.models.Transformer import TransformerModel
from vtbench.models.TCN import Small_TCN

from vtbench.CNN_train import create_dataloaders, train_model, evaluate_model, plot_class_distribution

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# File paths
train_file = 'data/ECG5000/ECG5000_TRAIN.ts'
test_file = 'data/ECG5000/ECG5000_TEST.ts'

# Load datasets
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
unique_labels = np.unique(y_train)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

nb_classes = len(unique_labels)

# Normalize features
x_train, x_test = normalize_data(x_train, x_test)

# Apply SMOTE with custom sampling strategy to training data
desired_samples_per_class = {  
    1: 200,
    2: 100,
    3: 50,
    4: 50
}
x_train_resampled, y_train_resampled = apply_smote(x_train, y_train, desired_samples_per_class)

# Convert to PyTorch tensors
X_train, y_train, X_test, y_test = to_torch_tensors(x_train_resampled, y_train_resampled, x_test, y_test)

# X_train, y_train, X_test, y_test = to_torch_tensors(x_train, y_train, x_test, y_test)


print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")


# Create DataLoaders
dataloaders = create_dataloaders(X_train, y_train, X_test, y_test)

for combo_key, loaders in dataloaders.items():
    print(f"Running model with combination: {combo_key}")
    
    train_loader, val_loader, test_loader = loaders

    # Initialize the 2DCNN model
    model = Simple2DCNN(3, nb_classes).to(device)  

    # Train the model
    num_epochs = 100
    train_model(model, train_loader, val_loader, num_epochs)

    # Evaluate the model
    evaluate_model(model, test_loader)

    print(f"\nFinished training for combination: {combo_key}\n" + "="*50 + "\n")




Loaded 500 samples from data/ECG5000/ECG5000_TRAIN.ts
Loaded 3910 samples from data/ECG5000/ECG5000_TEST.ts
Shape of X_train: torch.Size([692, 139])
Shape of y_train: torch.Size([692])
Shape of X_test: torch.Size([3910, 139])
Shape of y_test: torch.Size([3910])

Running model with chart_type: area, color_mode: color, label_mode: with_label

Running model with chart_type: area, color_mode: color, label_mode: without_label

Running model with chart_type: area, color_mode: monochrome, label_mode: with_label

Running model with chart_type: area, color_mode: monochrome, label_mode: without_label

Running model with chart_type: line, color_mode: color, label_mode: with_label

Running model with chart_type: line, color_mode: color, label_mode: without_label

Running model with chart_type: line, color_mode: monochrome, label_mode: with_label

Running model with chart_type: line, color_mode: monochrome, label_mode: without_label

Running model with chart_type: bar, bar_mode: fill, color_mode: c

C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/100], Loss: 1.2466, Training Accuracy: 51.16%
Validation Loss: 1.5024, Validation Accuracy: 39.28%
Epoch [2/100], Loss: 0.7502, Training Accuracy: 69.94%
Validation Loss: 0.4833, Validation Accuracy: 91.87%
Epoch [3/100], Loss: 0.5981, Training Accuracy: 74.57%
Validation Loss: 0.2960, Validation Accuracy: 92.53%
Epoch [4/100], Loss: 0.4682, Training Accuracy: 84.10%
Validation Loss: 0.2415, Validation Accuracy: 92.53%
Epoch [5/100], Loss: 0.2854, Training Accuracy: 91.47%
Validation Loss: 0.2342, Validation Accuracy: 93.20%
Epoch [6/100], Loss: 0.1786, Training Accuracy: 93.79%
Validation Loss: 0.5124, Validation Accuracy: 89.21%
Epoch [7/100], Loss: 0.1495, Training Accuracy: 95.66%
Validation Loss: 0.2953, Validation Accuracy: 92.84%
Epoch [8/100], Loss: 0.1852, Training Accuracy: 92.92%
Validation Loss: 0.3098, Validation Accuracy: 93.50%
Epoch [9/100], Loss: 0.1390, Training Accuracy: 96.39%
Validation Loss: 0.2824, Validation Accuracy: 93.30%
Epoch [10/100], Loss: 0.0828

C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/100], Loss: 1.2203, Training Accuracy: 52.46%
Validation Loss: 1.4340, Validation Accuracy: 83.73%
Epoch [2/100], Loss: 0.7205, Training Accuracy: 70.52%
Validation Loss: 0.5145, Validation Accuracy: 91.36%
Epoch [3/100], Loss: 0.5664, Training Accuracy: 76.73%
Validation Loss: 0.4012, Validation Accuracy: 88.29%
Epoch [4/100], Loss: 0.3574, Training Accuracy: 87.57%
Validation Loss: 0.3269, Validation Accuracy: 90.79%
Epoch [5/100], Loss: 0.2179, Training Accuracy: 92.49%
Validation Loss: 0.2966, Validation Accuracy: 91.05%
Epoch [6/100], Loss: 0.1783, Training Accuracy: 94.22%
Validation Loss: 0.2704, Validation Accuracy: 92.63%
Epoch [7/100], Loss: 0.1228, Training Accuracy: 94.80%
Validation Loss: 0.2749, Validation Accuracy: 92.94%
Epoch [8/100], Loss: 0.0938, Training Accuracy: 97.54%
Validation Loss: 0.7929, Validation Accuracy: 79.08%
Epoch [9/100], Loss: 0.1007, Training Accuracy: 96.68%
Validation Loss: 0.3672, Validation Accuracy: 91.15%
Epoch [10/100], Loss: 0.0580

C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/100], Loss: 1.2256, Training Accuracy: 50.87%
Validation Loss: 1.3224, Validation Accuracy: 89.82%
Epoch [2/100], Loss: 0.7537, Training Accuracy: 69.94%
Validation Loss: 0.4523, Validation Accuracy: 92.02%
Epoch [3/100], Loss: 0.6181, Training Accuracy: 73.12%
Validation Loss: 0.3604, Validation Accuracy: 92.38%
Epoch [4/100], Loss: 0.3688, Training Accuracy: 86.99%
Validation Loss: 0.2309, Validation Accuracy: 92.79%
Epoch [5/100], Loss: 0.2571, Training Accuracy: 89.88%
Validation Loss: 0.2940, Validation Accuracy: 91.76%
Epoch [6/100], Loss: 0.1764, Training Accuracy: 92.92%
